In [131]:
import sympy as sp
from IPython.display import display, Math

In [171]:
def generatePoly (n):
    
    symbols = []
    for i in range(n+1):
      symbols.append(sp.symbols(f"a_{i}"))
    
    s = sp.symbols('s')
    result = sp.poly(0, s)
    for x in range(n+1):
        result = result.add(sp.poly(symbols[x]*s**x,s))
    return result, symbols

In [172]:
w, s = sp.symbols('w s')

In [173]:
a, b, c, k = sp.symbols('a b c k', positive=True)

In [174]:
H, f, Dem, p, np, result = sp.symbols('H, f, Dem p np result', cls=sp.Function)

In [175]:
Dem = sp.poly(4*s**4 + 4*s**2 + 2, s) 
Dem

Poly(4*s**4 + 4*s**2 + 2, s, domain='ZZ')

In [177]:
p2 = generatePoly(2)[0]
p2

Poly(a_2*s**2 + a_1*s + a_0, s, domain='ZZ[a_0,a_1,a_2]')

In [99]:
np2 = sp.poly(p2.subs({s:-s}))
np2

Poly(a_2*(-s)**2 + a_1*(-s) + a_0, -s, domain='ZZ[a_0,a_1,a_2]')

In [178]:
result = sp.poly(p2 * np2, s)
result

Poly(a_2**2*s**4 + (2*a_0*a_2 - a_1**2)*s**2 + a_0**2, s, domain='ZZ[a_0,a_1,a_2]')

In [182]:
def coefArray (poly, dem):
    result = []
    n = sp.degree(dem)
    for i in range(n+1):
        result.append(poly.coeff_monomial(s**i) - dem.coeff_monomial(s**i))
    return result

In [183]:
coefarray = coefArray(result, Dem)
for x in coefarray:
    print(display(Math(sp.latex(x))))

<IPython.core.display.Math object>

None


<IPython.core.display.Math object>

None


<IPython.core.display.Math object>

None


<IPython.core.display.Math object>

None


<IPython.core.display.Math object>

None


In [184]:
coefarray

[a_0**2 - 2, 0, 2*a_0*a_2 - a_1**2 - 4, 0, a_2**2 - 4]

In [185]:
soluciones = sp.nonlinsolve(coefarray, generatePoly(2)[1])

In [186]:
def hasProperty(solucion):
    if(solucion[0].is_real and solucion[1].is_real and solucion[2].is_real):
        return sp.GreaterThan(solucion[0], 0) and sp.GreaterThan(solucion[1], 0) and sp.GreaterThan
(solucion[2], 0)

NameError: name 'solucion' is not defined

In [187]:
list(filter(hasProperty, soluciones))[0]

(sqrt(2), sqrt(-4 + 4*sqrt(2)), 2)